# demo

07 dec 2023  
sama ahmed

In [1]:
import numpy as np
from nre import io, moco
import ants

## hyperparameters

In [2]:
nifty_path = 'data/TSeries-12012023-1437-028_channel_1.nii'


## load data

In [3]:
# load xyzt
brain = io.load(nifty_path)

mean_brain = np.mean(brain, axis=-1)

## motion correction

Uses SyN transformation to warp each timepoint to a fixed brain

In [4]:
moco_brain = np.zeros_like(brain)

fixed = ants.from_numpy(mean_brain)
for tt in range(brain.shape[-1]):
    moving = ants.from_numpy(brain[:, :, :, tt])
    moco_brain[:, :, :, tt] = moco.apply(fixed, moving).numpy()

In [5]:
moco_brain.shape

(256, 256, 25, 10)